In [ ]:

!pip install transformers datasets evaluate sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install bert_score sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 11.5 MB/s eta 0:00:00


In [ ]:


import pandas as pd
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from evaluate import load as load_metric
from sentence_transformers import SentenceTransformer, util
import numpy as np


In [ ]:

faq_df = pd.read_excel("/content/WebScarpedDataset_for_DeepSeekchatbot_training.xlsx")
faq_df = faq_df.dropna(subset=["Question", "Answer", "Source URL"]).reset_index(drop=True)

# Sample 1–2 questions per unique source URL
sampled_df = faq_df.groupby("Source URL").apply(lambda x: x.sample(min(len(x), 2))).reset_index(drop=True)

# Load base model & tokenizer
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
model.eval()

/tmp/ipython-input-4-3843327530.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = faq_df.groupby("Source URL").apply(lambda x: x.sample(min(len(x), 2))).reset_index(drop=True)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [ ]:

# Load base model & tokenizer
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map="auto")
model.eval()

# Load evaluation metrics
rouge = load_metric("rouge")
bleu = load_metric("bleu")
bertscore = load_metric("bertscore")
sbert = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:

predictions, references, questions, source_urls = [], [], [], []

for _, row in sampled_df.iterrows():
    question = row["Question"]
    reference_answer = row["Answer"]
    source_url = row["Source URL"]

    input_text = f"Input: {question}\nOutput:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Output:" in generated:
        generated = generated.split("Output:")[-1].strip()

    predictions.append(generated)
    references.append(reference_answer)
    questions.append(question)
    source_urls.append(source_url)

In [ ]:

bleu_score = bleu.compute(predictions=predictions, references=references)

rouge_score = rouge.compute(predictions=predictions, references=references)

bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

semantic_sim = [util.cos_sim(sbert.encode(p), sbert.encode(r)).item() for p, r in zip(predictions, references)]



🔍 Evaluation on Sampled Base Model Answers:
Number of samples evaluated: 213
BLEU: 0.0310
ROUGE-1: 0.2067
ROUGE-2: 0.0496
ROUGE-L: 0.1432
BERTScore (F1 mean): 0.8280
Semantic Similarity (SBERT cosine avg): 0.5539

---
🔹 Q1: Who should use Amazon MQ?
🌐 Source: https://aws.amazon.com/amazon-mq/faqs/
🧠 Predicted: The options are A) A company with a large number of customers, B) A company with a large number of employees, C) A company with a large number of products, D) A company with a large number of employees.

Wait, the original question is asking who should use Amazon MQ. The options are A to D, each referring to a different type of company.

Hmm, I remember that Amazon MQ is a distributed messaging service. So, it's used for sending messages across a network
✅ Reference: Amazon MQ is suitable for enterprise IT professionals, developers, and architects who plan to adopt industry standard messaging APIs and protocols to decouple and scale their applications on the cloud without the op

In [ ]:

print("\n🔍 Evaluation on Sampled Base Model Answers:")
print(f"Number of samples evaluated: {len(predictions)}")
print(f"BLEU: {bleu_score['bleu']:.4f}")
print(f"ROUGE-1: {rouge_score['rouge1']:.4f}")
print(f"ROUGE-2: {rouge_score['rouge2']:.4f}")
print(f"ROUGE-L: {rouge_score['rougeL']:.4f}")
print(f"BERTScore (F1 mean): {np.mean(bertscore_result['f1']):.4f}")
print(f"Semantic Similarity (SBERT cosine avg): {np.mean(semantic_sim):.4f}")




🔍 Evaluation on Sampled Base Model Answers:
Number of samples evaluated: 213
BLEU: 0.0310
ROUGE-1: 0.2067
ROUGE-2: 0.0496
ROUGE-L: 0.1432
BERTScore (F1 mean): 0.8280
Semantic Similarity (SBERT cosine avg): 0.5539


In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    "Question": questions,
    "Predicted Answer": predictions,
    "Reference Answer": references,
    "Source URL": source_urls
})

results_df.to_excel("evaluation_results.xlsx", index=False)

print("Evaluation results exported to evaluation_results.xlsx")

Evaluation results exported to evaluation_results.xlsx
